In [1]:
import numpy as np
import pandas as pd

# model
from sklearn.metrics import log_loss

# NN
from keras.models import Sequential
from keras.layers import Dense

wd = '/Users/ewenwang/Documents/practice_data/conversion_rate/'
test_file = ['round1_ijcai_18_test_a_20180301.txt', 'round1_ijcai_18_test_b_20180418.txt']

file_UISC_union = 'snow_data_UISC_union.txt'
cvr_sm_union = 'cvr_bayesianSM_union.txt'
pagerank_union = 'pagerank_union.txt'

data = pd.read_csv(wd+file_UISC_union, sep=' ')
data = data.merge(pd.read_csv(wd+pagerank_union, sep=' '), on='instance_id', how='left').merge(pd.read_csv(wd+cvr_sm_union, sep=' '), on='instance_id', how='left')

Using TensorFlow backend.


In [7]:
train= data[(data['day'] >= 18) & (data['day'] <= 23)]
test= data[(data['day'] == 24)]

target = 'is_trade'
drop_list = ['is_trade', 'item_category_list', 'item_property_list', 'predict_category_property', 'realtime', 'context_timestamp', 'Unnamed: 0']
features = [x for x in data.columns if x not in drop_list]

X_train = train[features].fillna(0)
y_train = train[target].fillna(0)
X_test = test[features].fillna(0)
y_test = test[target].fillna(0)

In [21]:
print('shape of train X: ', X_train.shape)
print('shape of test X: ', X_test.shape)

shape of train X:  (422660, 333)
shape of test X:  (57588, 333)


In [34]:
model = Sequential()
model.add(Dense(2048, 
                input_dim=X_train.shape[1], 
                kernel_initializer='normal', 
                activation='relu'))
model.add(Dense(1024,
                kernel_initializer='normal', 
                activation='relu'))
model.add(Dense(512,
                kernel_initializer='normal', 
                activation='relu'))
model.add(Dense(16,
                kernel_initializer='normal', 
                activation='relu'))
model.add(Dense(1, 
                kernel_initializer='normal', 
                activation='sigmoid'))
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=5, batch_size=500)

Epoch 1/5
422660/422660 [==============================] - 785s 2ms/step - loss: 0.3481 - acc: 0.9784
Epoch 2/5
422660/422660 [==============================] - 11538s 27ms/step - loss: 0.3481 - acc: 0.9784
Epoch 3/5
 68000/422660 [===>..........................] - ETA: 3:40 - loss: 0.3397 - acc: 0.9789

KeyboardInterrupt: 

In [20]:
pred = model.predict(X_test)
test['pred'] = pred
test['index'] = range(len(test))
print('log loss ', log_loss(test['is_trade'], test['pred']))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


log loss  0.6363416294708114


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
